# Cash/3Mスプレッド分析 3: 相関・共和分分析

## 概要
本ノートブックでは、LME銅のCash/3Mスプレッドと関連市場変数との相関関係、および共和分関係を分析します。
スプレッド取引戦略の開発に向けた統計的基礎を構築します。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import psycopg2
from sqlalchemy import create_engine
from datetime import datetime, timedelta
import warnings
from scipy import stats
from statsmodels.tsa.stattools import coint, adfuller
from statsmodels.api import OLS
import statsmodels.api as sm
import os
from dotenv import load_dotenv

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10

print("ライブラリのインポートが完了しました")

In [ ]:
# データベース接続とCash/3Mスプレッドデータの読み込み
load_dotenv()

db_config = {
    'host': os.getenv('DB_HOST', 'localhost'),
    'database': os.getenv('DB_NAME', 'lme_copper_db'),
    'user': os.getenv('DB_USER', 'postgres'),
    'password': os.getenv('DB_PASSWORD', 'password'),
    'port': os.getenv('DB_PORT', '5432')
}

connection_string = f"postgresql://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}"
engine = create_engine(connection_string)

try:
    # まず接続テスト
    test_query = "SELECT 1"
    test_result = pd.read_sql_query(test_query, engine)
    print("データベース接続テスト成功")
    
    # Cash価格と3M価格からスプレッドを計算
    query = """
    WITH cash_data AS (
        SELECT trade_date, close_price as cash_price
        FROM lme_copper_futures
        WHERE ric = 'CMCU0' AND close_price IS NOT NULL
    ),
    three_month_data AS (
        SELECT trade_date, close_price as three_month_price
        FROM lme_copper_futures
        WHERE ric = 'CMCU3' AND close_price IS NOT NULL
    )
    SELECT 
        c.trade_date,
        c.cash_price,
        t.three_month_price,
        (c.cash_price - t.three_month_price) as cash_3m_spread
    FROM cash_data c
    INNER JOIN three_month_data t ON c.trade_date = t.trade_date
    ORDER BY c.trade_date
    """
    
    data = pd.read_sql_query(query, engine)
    
    if data.empty:
        raise Exception("データベースからデータを取得できませんでした")
    
    data['trade_date'] = pd.to_datetime(data['trade_date'])
    data.set_index('trade_date', inplace=True)
    
    print(f"データ読み込み成功: {len(data):,}件のレコード")
    
except Exception as e:
    print(f"❌ データベース接続エラー: {e}")
    print("\n🚨 CRITICAL ERROR: 実データの取得に完全に失敗しました")
    print("\n【データベース診断】")
    print("1. データベース接続: テスト実行")
    print("2. 必要なテーブル: lme_copper_futures") 
    print("3. 必要なRIC: CMCU0 (Cash), CMCU3 (3M)")
    print("\n【解決方法】")
    print("- データベースにCash/3Mの価格データが存在するか確認してください")
    print("- 以下のコマンドでデータを確認:")
    print("  SELECT COUNT(*) FROM lme_copper_futures WHERE ric IN ('CMCU0', 'CMCU3');")
    print("- データ収集スクリプトを実行してデータを追加してください")
    print("\n【トラブルシューティング】")
    print("1. データベースが起動しているか確認:")
    print("   brew services list | grep postgresql")
    print("2. 接続設定を確認:")
    print("   データベース名: lme_copper_db")
    print("   ユーザー: postgres")
    print("   ホスト: localhost")
    print("3. テーブルの存在を確認:")
    print("   psql -h localhost -U postgres -d lme_copper_db -c '\\dt'")
    
    # エラー時はNoneを設定して以降の処理を停止
    data = None

if data is not None:
    print("\n基本統計:")
    print(data.describe())
else:
    print("❌ データの読み込みに失敗しました。以降の分析を中止します。")
    print("上記のトラブルシューティング手順に従ってデータベースとデータを確認してください。")

In [ ]:
# 相関分析
correlation_matrix = data.corr()
print("相関行列:")
print(correlation_matrix)

# 相関の可視化
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, square=True)
plt.title('Cash/3M Spread Correlation Matrix')
plt.tight_layout()
plt.show()

# スプレッドと各価格の散布図
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].scatter(data['cash_price'], data['cash_3m_spread'], alpha=0.5)
axes[0].set_xlabel('Cash Price')
axes[0].set_ylabel('Cash/3M Spread')
axes[0].set_title('Spread vs Cash Price')
axes[0].grid(True, alpha=0.3)

axes[1].scatter(data['three_month_price'], data['cash_3m_spread'], alpha=0.5)
axes[1].set_xlabel('3M Price')
axes[1].set_ylabel('Cash/3M Spread')
axes[1].set_title('Spread vs 3M Price')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# 共和分テスト
print("=== 共和分分析 ===")

# Cash価格と3M価格の共和分テスト
try:
    coint_stat, p_value, crit_values = coint(data['cash_price'], data['three_month_price'])
    
    print("\nCash価格と3M価格の共和分テスト:")
    print(f"検定統計量: {coint_stat:.4f}")
    print(f"p値: {p_value:.4f}")
    print(f"臨界値:")
    print(f"  1%: {crit_values[0]:.4f}")
    print(f"  5%: {crit_values[1]:.4f}")
    print(f"  10%: {crit_values[2]:.4f}")
    
    is_cointegrated = p_value < 0.05
    print(f"\n共和分関係: {'あり' if is_cointegrated else 'なし'} (5%水準)")
    
    if is_cointegrated:
        # ヘッジ比率の計算
        X = sm.add_constant(data['three_month_price'])
        model = OLS(data['cash_price'], X).fit()
        hedge_ratio = model.params[1]
        intercept = model.params[0]
        
        print(f"\nヘッジ比率: {hedge_ratio:.4f}")
        print(f"切片: {intercept:.4f}")
        
        # スプレッド（誤差修正項）の計算
        spread = data['cash_price'] - hedge_ratio * data['three_month_price']
        
        # スプレッドの定常性テスト
        adf_stat, adf_pvalue, _, _, _, _ = adfuller(spread.dropna())
        print(f"\nスプレッドの定常性テスト (ADF):")
        print(f"検定統計量: {adf_stat:.4f}")
        print(f"p値: {adf_pvalue:.4f}")
        print(f"スプレッドは{'定常' if adf_pvalue < 0.05 else '非定常'}")
        
except Exception as e:
    print(f"共和分テストエラー: {e}")
    is_cointegrated = False

In [ ]:
# 共和分が確認された場合の可視化
if 'is_cointegrated' in locals() and is_cointegrated and 'spread' in locals():
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle('Cointegration Analysis Results', fontsize=16)
    
    # 1. 価格の時系列
    axes[0, 0].plot(data.index, data['cash_price'], label='Cash Price', alpha=0.8)
    axes[0, 0].plot(data.index, data['three_month_price'], label='3M Price', alpha=0.8)
    axes[0, 0].set_ylabel('Price')
    axes[0, 0].set_title('Price Series')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. 散布図と回帰線
    axes[0, 1].scatter(data['three_month_price'], data['cash_price'], alpha=0.5)
    x_range = np.linspace(data['three_month_price'].min(), data['three_month_price'].max(), 100)
    y_pred = intercept + hedge_ratio * x_range
    axes[0, 1].plot(x_range, y_pred, 'r-', linewidth=2, 
                   label=f'Hedge Ratio: {hedge_ratio:.3f}')
    axes[0, 1].set_xlabel('3M Price')
    axes[0, 1].set_ylabel('Cash Price')
    axes[0, 1].set_title('Price Relationship')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. スプレッドの時系列
    axes[1, 0].plot(spread.index, spread, color='purple', alpha=0.7)
    axes[1, 0].axhline(spread.mean(), color='red', linestyle='--', label='Mean')
    axes[1, 0].axhline(spread.mean() + 2*spread.std(), color='orange', linestyle='--', label='±2σ')
    axes[1, 0].axhline(spread.mean() - 2*spread.std(), color='orange', linestyle='--')
    axes[1, 0].set_ylabel('Spread')
    axes[1, 0].set_title('Cointegration Spread')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # 4. スプレッドの分布
    axes[1, 1].hist(spread.dropna(), bins=50, alpha=0.7, density=True)
    axes[1, 1].axvline(spread.mean(), color='red', linestyle='--', label='Mean')
    axes[1, 1].set_xlabel('Spread Value')
    axes[1, 1].set_ylabel('Density')
    axes[1, 1].set_title('Spread Distribution')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # スプレッドの基本統計
    print("\n共和分スプレッドの統計:")
    print(spread.describe())
    
else:
    print("共和分関係が確認されなかったため、通常のCash/3Mスプレッドを分析します。")
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Cash/3Mスプレッドの時系列
    axes[0].plot(data.index, data['cash_3m_spread'], color='blue', alpha=0.7)
    axes[0].axhline(data['cash_3m_spread'].mean(), color='red', linestyle='--', label='Mean')
    axes[0].set_ylabel('Spread (USD/MT)')
    axes[0].set_title('Cash/3M Spread Time Series')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # スプレッドの分布
    axes[1].hist(data['cash_3m_spread'], bins=50, alpha=0.7, density=True)
    axes[1].axvline(data['cash_3m_spread'].mean(), color='red', linestyle='--', label='Mean')
    axes[1].set_xlabel('Spread (USD/MT)')
    axes[1].set_ylabel('Density')
    axes[1].set_title('Spread Distribution')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# 簡単なペアトレーディング戦略の検証
print("=== ペアトレーディング戦略分析 ===")

# 適切なスプレッドを選択
if 'spread' in locals() and is_cointegrated:
    trading_spread = spread
    strategy_name = "共和分ベース"
else:
    trading_spread = data['cash_3m_spread']
    strategy_name = "Cash/3Mスプレッド"

# 移動平均とボリンジャーバンドの計算
window = 60
spread_ma = trading_spread.rolling(window=window).mean()
spread_std = trading_spread.rolling(window=window).std()
upper_band = spread_ma + 2 * spread_std
lower_band = spread_ma - 2 * spread_std

# Zスコアの計算
z_score = (trading_spread - spread_ma) / spread_std

# 取引シグナルの生成
positions = pd.Series(0, index=trading_spread.index)
positions[z_score < -2] = 1  # スプレッド買い
positions[z_score > 2] = -1  # スプレッド売り
positions[abs(z_score) < 0.5] = 0  # ポジション解消

# ポジションをフォワードフィル
positions = positions.replace(0, np.nan).fillna(method='ffill').fillna(0)

# 戦略リターンの計算
spread_returns = trading_spread.diff()
strategy_returns = positions.shift(1) * spread_returns

# パフォーマンス指標
cumulative_returns = strategy_returns.cumsum()
total_return = cumulative_returns.iloc[-1]
volatility = strategy_returns.std() * np.sqrt(252)
sharpe_ratio = strategy_returns.mean() / strategy_returns.std() * np.sqrt(252)
max_drawdown = (cumulative_returns - cumulative_returns.cummax()).min()

print(f"\n{strategy_name}戦略のパフォーマンス:")
print(f"総リターン: {total_return:.2f}")
print(f"ボラティリティ (年率): {volatility:.2f}")
print(f"シャープレシオ: {sharpe_ratio:.3f}")
print(f"最大ドローダウン: {max_drawdown:.2f}")

# 取引頻度
position_changes = positions.diff().abs().sum()
print(f"ポジション変更回数: {position_changes/2:.0f}回")

In [ ]:
# 戦略結果の可視化
fig, axes = plt.subplots(3, 1, figsize=(14, 12))
fig.suptitle(f'{strategy_name} ペアトレーディング戦略', fontsize=16)

# 1. スプレッドとボリンジャーバンド
axes[0].plot(trading_spread.index, trading_spread, label='Spread', alpha=0.7)
axes[0].plot(spread_ma.index, spread_ma, 'r--', label='Moving Average')
axes[0].plot(upper_band.index, upper_band, 'g--', alpha=0.7, label='Upper Band (+2σ)')
axes[0].plot(lower_band.index, lower_band, 'g--', alpha=0.7, label='Lower Band (-2σ)')
axes[0].fill_between(upper_band.index, lower_band, upper_band, alpha=0.2, color='gray')
axes[0].set_ylabel('Spread Value')
axes[0].set_title('Spread with Bollinger Bands')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 2. Zスコアとポジション
ax2_1 = axes[1]
ax2_1.plot(z_score.index, z_score, 'b-', alpha=0.7, label='Z-Score')
ax2_1.axhline(2, color='red', linestyle='--', alpha=0.5, label='Entry Threshold')
ax2_1.axhline(-2, color='red', linestyle='--', alpha=0.5)
ax2_1.axhline(0.5, color='green', linestyle='--', alpha=0.5, label='Exit Threshold')
ax2_1.axhline(-0.5, color='green', linestyle='--', alpha=0.5)
ax2_1.set_ylabel('Z-Score')
ax2_1.legend(loc='upper left')

# ポジションを別軸で表示
ax2_2 = ax2_1.twinx()
ax2_2.fill_between(positions.index, 0, positions, where=positions>0, 
                  alpha=0.3, color='green', label='Long Position')
ax2_2.fill_between(positions.index, 0, positions, where=positions<0, 
                  alpha=0.3, color='red', label='Short Position')
ax2_2.set_ylabel('Position')
ax2_2.legend(loc='upper right')
ax2_1.set_title('Z-Score and Trading Positions')
ax2_1.grid(True, alpha=0.3)

# 3. 累積リターン
axes[2].plot(cumulative_returns.index, cumulative_returns, 'g-', linewidth=2, label='Strategy Returns')
axes[2].axhline(0, color='black', linestyle='-', alpha=0.3)
axes[2].set_ylabel('Cumulative Returns')
axes[2].set_title('Strategy Performance')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

# パフォーマンス統計をテキストで追加
stats_text = f"Total Return: {total_return:.2f}\n"
stats_text += f"Sharpe Ratio: {sharpe_ratio:.3f}\n"
stats_text += f"Max Drawdown: {max_drawdown:.2f}"
axes[2].text(0.02, 0.95, stats_text, transform=axes[2].transAxes, 
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5),
            verticalalignment='top')

plt.tight_layout()
plt.show()

print("\n=== 分析結果まとめ ===")
print(f"データ期間: {data.index.min().strftime('%Y-%m-%d')} ～ {data.index.max().strftime('%Y-%m-%d')}")
print(f"総観測数: {len(data):,}件")
print(f"\nCash/3Mスプレッド統計:")
print(f"平均: {data['cash_3m_spread'].mean():.2f}")
print(f"標準偏差: {data['cash_3m_spread'].std():.2f}")
print(f"最大値: {data['cash_3m_spread'].max():.2f}")
print(f"最小値: {data['cash_3m_spread'].min():.2f}")

if 'is_cointegrated' in locals():
    print(f"\n共和分関係: {'確認' if is_cointegrated else '未確認'}")
    if is_cointegrated:
        print(f"ヘッジ比率: {hedge_ratio:.4f}")

print(f"\n戦略提案:")
if is_cointegrated:
    print("- 共和分ベースのペアトレーディング戦略が有効")
    print("- Zスコア±2σでエントリー、±0.5σでエグジット")
    print("- 平均回帰特性を活用した取引")
else:
    print("- 標準的なCash/3Mスプレッド取引")
    print("- ボリンジャーバンドベースの逆張り戦略")
    print("- トレンドフォロー要素も検討")